In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from config_vacation import g_key

In [2]:
#reading csv from part I
cities_data = '../WeatherPy/output_data/cities.csv'
cities_df = pd.read_csv(cities_data)

set_cities_df = cities_df.set_index('City_ID')
set_cities_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
City_ID,,,,,,,,,
0,hermanus,-34.4187,19.2345,64.99,94,98,11.48,ZA,1612674939
1,cabo san lucas,22.8909,-109.9124,64.40,59,20,5.75,MX,1612674989
2,avarua,-21.2078,-159.7750,82.40,74,7,11.50,CK,1612674485
3,libohove,40.0311,20.2631,59.00,100,20,4.85,AL,1612675020
4,saint-philippe,-21.3585,55.7679,84.20,70,0,8.05,RE,1612675020
...,...,...,...,...,...,...,...,...,...
575,baghdad,33.3406,44.4009,55.40,94,75,6.91,IQ,1612675057
576,kapuskasing,49.4169,-82.4331,-11.20,77,30,3.44,CA,1612675235
577,ararat,39.8317,44.7049,28.40,93,0,1.14,AM,1612675235


In [3]:
set_cities_df['Humidity'].max()

100

In [4]:
#humidity heatmap
gmaps.configure(api_key = g_key)

locations = set_cities_df[['Lat', 'Lng']]
fig = gmaps.figure(map_type= 'HYBRID')

humid = set_cities_df['Humidity']

heat_layer = gmaps.heatmap_layer(locations, weights = humid, 
                                 dissipating = False, max_intensity = set_cities_df['Humidity'].max(),
                                 point_radius = 3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#ideal weather conditions
#max temp = 80, but nothing less than 60
#wind speed less than 10
#humidity nothing greater than 50
#cloudiness less than 25

#dropping temp
ideal_temp_index = set_cities_df[(set_cities_df['Max_Temp'] > 80) | (set_cities_df['Max_Temp'] < 50)].index
ideal_temp = set_cities_df.drop(ideal_temp_index, inplace = False)

#dropping wind
ideal_wind_index = ideal_temp[ideal_temp['Wind_Speed'] > 10].index
ideal_temp_wind = ideal_temp.drop(ideal_wind_index, inplace = False)

#dropping humidity
ideal_humid_index = ideal_temp_wind[ideal_temp_wind['Humidity'] > 30].index 
ideal_temp_wind_humidity = ideal_temp_wind.drop(ideal_humid_index, inplace = False)

#dropping cloudiness
ideal_cloud_index = ideal_temp_wind_humidity[ideal_temp_wind_humidity['Cloudiness'] > 35].index
ideal_weather_df = ideal_temp_wind_humidity.drop(ideal_cloud_index, inplace = False)

ideal_weather_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
City_ID,,,,,,,,,
24,tezu,27.9167,96.1667,78.42,17,3,4.45,IN,1612675027
53,arua,3.0201,30.9111,65.21,27,18,5.39,UG,1612675038
173,joshimath,30.5667,79.5667,60.91,28,0,4.14,IN,1612674910
181,jimenez,28.3333,-105.4000,50.00,23,1,4.61,MX,1612675085
234,ejido hermosillo,32.5121,-114.9238,64.40,29,1,1.59,MX,1612675113
301,hasaki,35.7333,140.8333,64.00,25,20,5.75,JP,1612674662
327,teocuitatlan de corona,20.1167,-103.4000,62.01,26,1,2.53,MX,1612675144
358,kollo,13.3043,2.3390,71.60,21,24,3.44,NE,1612675158
465,kharan,28.5833,65.4167,66.51,17,0,0.51,PK,1612675194


In [6]:
#hotel df
hotel_df = ideal_weather_df
hotel_df['Hotel_Name'] = ""

In [7]:
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
City_ID,,,,,,,,,,
24,tezu,27.9167,96.1667,78.42,17,3,4.45,IN,1612675027,
53,arua,3.0201,30.9111,65.21,27,18,5.39,UG,1612675038,
173,joshimath,30.5667,79.5667,60.91,28,0,4.14,IN,1612674910,
181,jimenez,28.3333,-105.4000,50.00,23,1,4.61,MX,1612675085,
234,ejido hermosillo,32.5121,-114.9238,64.40,29,1,1.59,MX,1612675113,
301,hasaki,35.7333,140.8333,64.00,25,20,5.75,JP,1612674662,
327,teocuitatlan de corona,20.1167,-103.4000,62.01,26,1,2.53,MX,1612675144,
358,kollo,13.3043,2.3390,71.60,21,24,3.44,NE,1612675158,
465,kharan,28.5833,65.4167,66.51,17,0,0.51,PK,1612675194,


In [8]:
# geocoordinates
for i in range(len(hotel_df)):
    try:
        target_coordinates = f'{hotel_df.iloc[i,1]}, {hotel_df.iloc[i,2]}'
        params = {'location': target_coordinates,
                  'keyword': 'hotel',
                  'radius': 5000,
                  'key': g_key,}
        base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
        response = requests.get(base_url, params = params)
        hotel_data = response.json()
        print(hotel_data['results'][0]['name'])
        hotel_df.iloc[i, 9] = hotel_data['results'][0]['name']
    except:
        try: 
#wanted to add option of hostel in location where hotels may not be available
            target_coordinates = f'{hotel_df.iloc[i,1]}, {hotel_df.iloc[i,2]}'
            params = {'location': target_coordinates,
                      'keyword': 'hostel',
                      'radius': 5000,
                      'key': g_key,}
            base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
            response = requests.get(base_url, params = params)
            hostel_data = response.json()
            hotel_df.iloc[i, 9] = hostel_data['results'][0]['name']
        except:
            print('No hotel')
            hotel_df.iloc[i, 9] = 'No Hotel Found'


Hotel Shivam
Desert Breeze Hotel Arua
The Tattva - A Boutique Resort
No hotel
No hotel
ホテル＆スパ月美 太陽の里
No hotel
Mansoor Hotel & Restaurant
Alqadri hotel zero point ormara
Hotel Vedant Inn
No hotel


In [9]:
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
City_ID,,,,,,,,,,
24,tezu,27.9167,96.1667,78.42,17,3,4.45,IN,1612675027,Hotel Shivam
53,arua,3.0201,30.9111,65.21,27,18,5.39,UG,1612675038,Desert Breeze Hotel Arua
173,joshimath,30.5667,79.5667,60.91,28,0,4.14,IN,1612674910,The Tattva - A Boutique Resort
181,jimenez,28.3333,-105.4000,50.00,23,1,4.61,MX,1612675085,No Hotel Found
234,ejido hermosillo,32.5121,-114.9238,64.40,29,1,1.59,MX,1612675113,No Hotel Found
301,hasaki,35.7333,140.8333,64.00,25,20,5.75,JP,1612674662,ホテル＆スパ月美 太陽の里
327,teocuitatlan de corona,20.1167,-103.4000,62.01,26,1,2.53,MX,1612675144,Hotel Casona La Inmaculada
358,kollo,13.3043,2.3390,71.60,21,24,3.44,NE,1612675158,No Hotel Found
465,kharan,28.5833,65.4167,66.51,17,0,0.51,PK,1612675194,Mansoor Hotel & Restaurant


In [10]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
#adding marker so that when clicked it displays hotel info
marker_layer = gmaps.marker_layer(locations,
                                 info_box_content = hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))